In [1]:
import itk
from itk import TubeTK as tube

import numpy as np

In [2]:
InputBaseName = "../data/CT/CT"
OutputBaseName = "results/CT-Lungs"

InputFilename = OutputBaseName+".mha"

imLungs = itk.imread(InputFilename, itk.F)

In [3]:
ImageType = itk.Image[itk.F, 3]

spacing = imLungs.GetSpacing()[0]

# Erode imLungs to avoid noise at edges
imMath = tube.ImageMath.New(imLungs)
imMath.Threshold(-1025,-1025,0,1)
imMath.Erode(10,1,0)
imLungsMaskErode = imMath.GetOutput()
imMath.SetInput(imLungs)
imMath.IntensityWindow(-100,2048,0,1)
imMath.ReplaceValuesOutsideMaskRange(imLungsMaskErode,0.5,1.5,0)
imLungsErode = imMath.GetOutput()

imMath.SetInput(imLungsErode)
imMath.Blur(4*spacing)
imBlurBig = imMath.GetOutput()
imMath.SetInput(imLungs)
imMath.Blur(0.5*spacing)
imMath.AddImages(imBlurBig, 1, -1)
imDoG = imMath.GetOutput()

imDoGArray = itk.GetArrayFromImage(imDoG)

numSeeds = 15
seedCoverage = 20
seedCoord = np.zeros([numSeeds,3])
for i in range(numSeeds):
    seedCoord[i] = np.unravel_index(np.argmax(imDoGArray, axis=None), imDoGArray.shape)
    indx = [int(seedCoord[i][0]),int(seedCoord[i][1]),int(seedCoord[i][2])]
    minX = max(indx[0]-seedCoverage,0)
    maxX = max(indx[0]+seedCoverage,imDoGArray.shape[0])
    minY = max(indx[1]-seedCoverage,0)
    maxY = max(indx[1]+seedCoverage,imDoGArray.shape[1])
    minZ = max(indx[2]-seedCoverage,0)
    maxZ = max(indx[2]+seedCoverage,imDoGArray.shape[2])
    imDoGArray[minX:maxX,minY:maxY,minZ:maxZ]=-1024
    indx.reverse()
    seedCoord[:][i] = imLungs.TransformIndexToPhysicalPoint(indx)
print(seedCoord)

itkImageF4 not loaded from module TubeTK because of exception:
 module 'itk.TubeTKPython' has no attribute 'itkImageF4'
vectoritkImageF4 not loaded from module TubeTK because of exception:
 module 'itk.TubeTKPython' has no attribute 'vectoritkImageF4'


[[  56.88750452 -136.45000285   65.59679794]
 [ -64.3656224  -130.42500275  157.4780494 ]
 [  71.19687974 -143.98125297   48.27492267]
 [ -41.77187204 -156.78437817   54.29992276]
 [ -68.13124746 -149.25312805   49.78117269]
 [ -90.72499782 -207.24375397   69.362423  ]
 [ -73.40312254 -125.90625268   37.7311725 ]
 [ -93.73749787 -152.2656281    68.60929799]
 [ -92.98437285 -161.30312824   51.28742272]
 [-115.57812321 -165.82187831   33.21242243]
 [-106.54062307 -146.99375302   35.47179747]
 [ -37.25312197 -122.14062762   15.13742214]
 [-100.51562297 -114.6093775   158.98429942]
 [ -74.15624756 -110.09062743  140.90929914]
 [-103.52812302 -124.40000266  124.34054887]]


In [4]:
# Manually extract a few vessels to form an image-specific training set
vSeg = tube.SegmentTubes.New(Input=imLungs)
vSeg.SetVerbose(True)
vSeg.SetMinRoundness(0.4)
vSeg.SetMinRidgeness(0.8)
#vSeg.SetMinCurvature(0.02)
vSeg.SetRadiusInObjectSpace( 0.5 )
vSeg.SetMinLength(300)
for i in range(numSeeds):
    print("**** Processing seed " + str(i) + " : " + str(seedCoord[i]))
    vSeg.ExtractTubeInObjectSpace( seedCoord[i], i )
    
tubeMaskImage = vSeg.GetTubeMaskImage()

**** Processing seed 0 : [  56.88750452 -136.45000285   65.59679794]
**** Processing seed 1 : [ -64.3656224  -130.42500275  157.4780494 ]
**** Processing seed 2 : [  71.19687974 -143.98125297   48.27492267]
**** Processing seed 3 : [ -41.77187204 -156.78437817   54.29992276]
**** Processing seed 4 : [ -68.13124746 -149.25312805   49.78117269]
**** Processing seed 5 : [ -90.72499782 -207.24375397   69.362423  ]
**** Processing seed 6 : [ -73.40312254 -125.90625268   37.7311725 ]
**** Processing seed 7 : [ -93.73749787 -152.2656281    68.60929799]
**** Processing seed 8 : [ -92.98437285 -161.30312824   51.28742272]
**** Processing seed 9 : [-115.57812321 -165.82187831   33.21242243]
**** Processing seed 10 : [-106.54062307 -146.99375302   35.47179747]
**** Processing seed 11 : [ -37.25312197 -122.14062762   15.13742214]
**** Processing seed 12 : [-100.51562297 -114.6093775   158.98429942]
**** Processing seed 13 : [ -74.15624756 -110.09062743  140.90929914]
**** Processing seed 14 : [-10

In [5]:
itk.imwrite(tubeMaskImage, OutputBaseName+"-VesselsInitial.mha")

In [6]:
LabelMapType = itk.Image[itk.UC,3]

trMask = tube.ComputeTrainingMask[ImageType,LabelMapType].New()
trMask.SetInput( tubeMaskImage )
trMask.SetGap( 3 )
trMask.SetObjectWidth( 1 )
trMask.SetNotObjectWidth( 1 )
trMask.Update()
fgMask = trMask.GetOutput()

In [7]:
itk.imwrite(fgMask, OutputBaseName+"-VesselsInitialMask.mha")

In [8]:
enhancer = tube.EnhanceTubesUsingDiscriminantAnalysis[ImageType,LabelMapType].New()
enhancer.AddInput(imLungs)
enhancer.SetLabelMap(fgMask)
enhancer.SetRidgeId(255)
enhancer.SetBackgroundId(128)
enhancer.SetUnknownId(0)
enhancer.SetTrainClassifier(True)
enhancer.SetUseIntensityOnly(True)
enhancer.SetUseFeatureMath(True)
enhancer.SetScales([1*spacing,2*spacing,6*spacing]) #(size*0.5,size*1.5,size*3.5,size*5.5)
enhancer.Update()
enhancer.ClassifyImages()

In [9]:
imMath.SetInput(enhancer.GetClassProbabilityImage(0))
imMath.Blur(0.5*spacing)
prob0 = imMath.GetOutput()
imMath.SetInput(enhancer.GetClassProbabilityImage(1))
imMath.Blur(0.5*spacing)
prob1 = imMath.GetOutput()

imMath.SetInput(imLungs)
imMath.Threshold(-1025,-1025,0,1)
imMath.Erode(2,1,0)
imLungsE = imMath.GetOutput()

imMath.SetInput(prob0)
imMath.ReplaceValuesOutsideMaskRange(imLungsE,1,1,0)
prob0 = imMath.GetOutput()

imMath.SetInput(prob1)
imMath.ReplaceValuesOutsideMaskRange(imLungsE,1,1,0)
prob1 = imMath.GetOutput()

imDiff = itk.SubtractImageFilter(Input1=prob0, Input2=prob1)
imDiffArr = itk.GetArrayFromImage(imDiff)
dMax = imDiffArr.max()
imProbArr = imDiffArr / dMax
imLungsVess = itk.GetImageFromArray(imProbArr)
imLungsVess.CopyInformation(imLungs)

imMath.SetInput(imLungsVess)
imMath.ReplaceValuesOutsideMaskRange(imLungsE,1,1,-1)
imLungsVess = imMath.GetOutput()


In [10]:
itk.imwrite( imLungsVess, OutputBaseName+"-VesselsEnhanced.mha", compression=True)

In [11]:
imMath.SetInput(imLungs)
imMath.Threshold(-1025,-1025,0,1)
imMath.Erode(8,1,0)
imMaskedE = imMath.GetOutput()

imMath.SetInput(imLungsVess)
imMath.ReplaceValuesOutsideMaskRange(imMaskedE, 1, 1, -1)
imVessMasked = imMath.GetOutput()

In [12]:
itk.imwrite( imVessMasked, OutputBaseName+"-VesselsEnhanced-Masked.mha", compression=True)